In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

import matplotlib.patheffects as pe
import os
import matplotlib.pyplot as plt
import seaborn as sns
# Seaborn aesthetics
sns.set_context("notebook")
sns.set_theme(style="whitegrid", font_scale=1.4,
              rc={'grid.color': '#ededed'})

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn import linear_model
import statsmodels.api as sm
import scipy
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba

In [3]:
# Read the above saved file (if necessary)
neon_vst = pd.read_csv("../../data/output/NEON_VST_TREES.csv")
# neon_vst = pd.read_csv("../../data/output/NEON_VST_TREES_2020to2023.csv")
# Use only live trees
neon_vst_filt = neon_vst[neon_vst['plantStatus'].str.contains("Live", na=False)].reset_index(drop=True)
neon_vst_filt = neon_vst_filt[~neon_vst_filt['plantStatus'].str.contains("damage|broken", na=False)].reset_index(drop=True)

# Get average crownd diameter
neon_vst_filt["avgCrownDiameter"] = (neon_vst_filt["maxCrownDiameter"] + neon_vst_filt["ninetyCrownDiameter"])/2
# Get vertical crown diameter
neon_vst_filt["verticalDiameter"] = neon_vst_filt["height"] - neon_vst_filt["baseCrownHeight"]
# calculate crown volume 
hrad = neon_vst_filt["avgCrownDiameter"]/2
vrad = neon_vst_filt["verticalDiameter"]/2
# appidv_all['crown_vol'] =(4/3) * np.pi * (hrad**2) * (vrad)
neon_vst_filt['crown_vol'] =(4/3) * np.pi * (hrad**2) * ((1/4)*neon_vst_filt["height"])

In [4]:
## Clean data
# remember to divide by 100 for stemDiameter in meters!
neon_vst_filt['stemDiameter_m'] = neon_vst_filt['stemDiameter']/100
neon_vst_filt['crownRadius'] = neon_vst_filt['avgCrownDiameter']/2
neon_vst_filt['verticalCrownRadius'] = neon_vst_filt['verticalDiameter']/2

# Remove data with errors (large DBH, height, etc.)
treeid_list = ['NEON.PLA.D17.SOAP.05687']
neon_vst_clean = neon_vst_filt[~neon_vst_filt['individualID'].isin(treeid_list)].copy().reset_index(drop=True)
print(len(neon_vst_clean))

121823


In [5]:
#########

In [6]:
## Linear regression function
from scipy import stats
import numpy as np

def LinearRegression(x,y):
    """
    returns: regression model
    """
    filterbool = ~(np.isnan(x) | np.isnan(y))
    x = x[filterbool]
    y = y[filterbool]
    # Simple OLS linear regression using scipy stats, which is what statsmodel depends on
    try:
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    except:
        print(f"Not enough data at {siteid}. Length: ", len(x))
        return np.nan, np.nan, np.nan, np.nan, np.nan
    return slope, intercept, r_value, p_value, std_err


In [7]:
# init results df
results_df_h_d = pd.DataFrame({"siteID":neon_vst_clean['siteID'].unique()})
results_df_rc_d = pd.DataFrame({"siteID":neon_vst_clean['siteID'].unique()})
results_df_rc_h = pd.DataFrame({"siteID":neon_vst_clean['siteID'].unique()})
results_df_vc_h = pd.DataFrame({"siteID":neon_vst_clean['siteID'].unique()})



# Loop through each site, get regression, save scaling exponent
for siteid, site_df in neon_vst_clean.groupby("siteID"):
    print(siteid, end="\r")
    ## H ~ D
    x = np.log(site_df['stemDiameter'].values)
    y = np.log(site_df['height'].values)
    slope, intercept, r_value, p_value, std_err = LinearRegression(x,y)
    results_df_h_d.loc[results_df_h_d['siteID']==siteid, "slope"] = np.round(slope,3)
    results_df_h_d.loc[results_df_h_d['siteID']==siteid, "intercept"] = np.round(intercept,3)
    results_df_h_d.loc[results_df_h_d['siteID']==siteid, "R2"] = np.round(r_value**2,3)
    results_df_h_d.loc[results_df_h_d['siteID']==siteid, "p_value"] = np.round(p_value,4)
    results_df_h_d.loc[results_df_h_d['siteID']==siteid, "std_err"] = np.round(std_err,3)
    
    ## Rc ~ D
    x = np.log(site_df['stemDiameter'].values)
    y = np.log(site_df['crownRadius'].values)
    slope, intercept, r_value, p_value, std_err = LinearRegression(x,y)
    results_df_rc_d.loc[results_df_rc_d['siteID']==siteid, "slope"] = np.round(slope,3)
    results_df_rc_d.loc[results_df_rc_d['siteID']==siteid, "intercept"] = np.round(intercept,3)
    results_df_rc_d.loc[results_df_rc_d['siteID']==siteid, "R2"] = np.round(r_value**2,3)
    results_df_rc_d.loc[results_df_rc_d['siteID']==siteid, "p_value"] = np.round(p_value,4)
    results_df_rc_d.loc[results_df_rc_d['siteID']==siteid, "std_err"] = np.round(std_err,3)
    
    ## Rc ~ H
    x = np.log(site_df['height'].values)
    y = np.log(site_df['crownRadius'].values)
    slope, intercept, r_value, p_value, std_err = LinearRegression(x,y)
    results_df_rc_h.loc[results_df_rc_h['siteID']==siteid, "slope"] = np.round(slope,3)
    results_df_rc_h.loc[results_df_rc_h['siteID']==siteid, "intercept"] = np.round(intercept,3)
    results_df_rc_h.loc[results_df_rc_h['siteID']==siteid, "R2"] = np.round(r_value**2,3)
    results_df_rc_h.loc[results_df_rc_h['siteID']==siteid, "p_value"] = np.round(p_value,4)
    results_df_rc_h.loc[results_df_rc_h['siteID']==siteid, "std_err"] = np.round(std_err,3)
    
    ## Vc ~ H
    x = np.log(site_df['height'].values)
    y = np.log(site_df['crown_vol'].values)
    slope, intercept, r_value, p_value, std_err = LinearRegression(x,y)
    results_df_vc_h.loc[results_df_vc_h['siteID']==siteid, "slope"] = np.round(slope,3)
    results_df_vc_h.loc[results_df_vc_h['siteID']==siteid, "intercept"] = np.round(intercept,3)
    results_df_vc_h.loc[results_df_vc_h['siteID']==siteid, "R2"] = np.round(r_value**2,3)
    results_df_vc_h.loc[results_df_vc_h['siteID']==siteid, "p_value"] = np.round(p_value,4)
    results_df_vc_h.loc[results_df_vc_h['siteID']==siteid, "std_err"] = np.round(std_err,3)
    
    

Not enough data at SRER. Length:  0
Not enough data at SRER. Length:  2
Not enough data at SRER. Length:  2


In [8]:
# Lets save outputs!
results_df_h_d.to_csv("../../data/output/site-allometry/NEON_site_allom_H_D.csv",index=False)
results_df_rc_d.to_csv("../../data/output/site-allometry/NEON_site_allom_Rc_D.csv",index=False)
results_df_rc_h.to_csv("../../data/output/site-allometry/NEON_site_allom_Rc_H.csv",index=False)
results_df_vc_h.to_csv("../../data/output/site-allometry/NEON_site_allom_Vc_H.csv",index=False)